# Demo

This is a demonstration of the basic Python usage of the `anarci-toolz` package.

Please refer to the README for guidance on usage on the command line interface tool.

In [1]:
import pandas as pd
from anarci_toolz.pipeline import run_anarci_toolz

Fetch data from open source database of therapeutic antibodies from [TheraSabDab](https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/therasabdab/search/)

In [2]:
TARGET_URL = "https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/static/downloads/TheraSAbDab_SeqStruc_OnlineDownload.csv"
df = pd.read_csv(TARGET_URL)

In [3]:
# Do some light transformation to prepare data
df = df[df['Format']=="Whole mAb"]
n_sample = 100
df = df.sample(n_sample)
df = df[['Therapeutic', 'Format', 'HeavySequence', 'LightSequence']]

# Note: here we designate the amino acid sequence column as "sequence_aa"
df_long = pd.melt(df, id_vars=['Therapeutic', 'Format'], value_vars=['HeavySequence', 'LightSequence'], var_name='chain_type_input', value_name='sequence_aa')

In [4]:
df_long.head()

,Therapeutic,Format,chain_type_input,sequence_aa
0,Vantictumab,Whole mAb,HeavySequence,EVQLVESGGGLVQPGGSLRLSCAASGFTFSHYTLSWVRQAPGKGLE...
1,Sabatolimab,Whole mAb,HeavySequence,QVQLVQSGAEVKKPGSSVKVSCKASGYTFTSYNMHWVRQAPGQGLE...
2,Briquilimab,Whole mAb,HeavySequence,QVQLVQSGAEVKKPGASVKVSCKASGYTFTSYNMHWVRQAPGQGLE...
3,Narlumosbart,Whole mAb,HeavySequence,EVQLLESGGGLVQPGGSLRLSCAASGFTFSSYAMSWVRQAPGKGLE...
4,Varlilumab,Whole mAb,HeavySequence,QVQLVESGGGVVQPGRSLRLSCAASGFTFSSYDMHWVRQAPGKGLE...


In [5]:
# Run anarci-toolz

df_result = run_anarci_toolz(
    df=df_long,
    scheme="imgt",
    allowed_species=["human"],
    seq_aa_header="sequence_aa",)

2025-03-11 20:06:11.671 | INFO     | anarci_toolz.abnumber_tool:run_parallel_abnumber:62 - Starting AbNumber processing ...
Processing sequences: 100%|██████████| 200/200 [00:00<00:00, 43620.24seq/s]
2025-03-11 20:06:13.880 | SUCCESS  | anarci_toolz.abnumber_tool:parallel_get_region_seqs:138 - AbNumber parallel processing complete
2025-03-11 20:06:13.931 | INFO     | anarci_toolz.anarci_tool:run_parallel_anarci:91 - Starting ANARCI processing ...
Processing sequences: 100%|██████████| 200/200 [00:00<00:00, 66932.16seq/s]
2025-03-11 20:06:15.828 | SUCCESS  | anarci_toolz.anarci_tool:run_parallel_anarci:104 - ANARCI parallel processing complete
2025-03-11 20:06:15.858 | SUCCESS  | anarci_toolz.pipeline:run_anarci_toolz:86 - anarci-toolz run complete.


We can see a comprehensive annotated ANARCI result from the `anarci-toolz`:

In [6]:
df_result.head()

,Therapeutic,Format,chain_type_input,sequence_aa,scheme,passed_abnumber,sequence_alignment_aa,species,chain_type,v_gene,...,fr1_aa,fr2_aa,fr3_aa,fr4_aa,passed_anarci,variable_region_start_index,variable_region_end_index,e_value,bitscore,bias
0,Vantictumab,Whole mAb,HeavySequence,EVQLVESGGGLVQPGGSLRLSCAASGFTFSHYTLSWVRQAPGKGLE...,imgt,True,EVQLVESGGGLVQPGGSLRLSCAASGFTFSHYTLSWVRQAPGKGLE...,alpaca,H,IGHV3-23*04,...,EVQLVESGGGLVQPGGSLRLSCAAS,LSWVRQAPGKGLEWVSV,YYADSVKGRFTISSDNSKNTLYLQMNSLRAEDTAVYYC,WGQGTLVTVSS,True,0,118,5.600000e-64,205.3,2.1
1,Sabatolimab,Whole mAb,HeavySequence,QVQLVQSGAEVKKPGSSVKVSCKASGYTFTSYNMHWVRQAPGQGLE...,imgt,True,QVQLVQSGAEVKKPGSSVKVSCKASGYTFTSYNMHWVRQAPGQGLE...,human,H,IGHV1-46*01,...,QVQLVQSGAEVKKPGSSVKVSCKAS,MHWVRQAPGQGLEWMGD,SYNQKFKGRVTITADKSTSTVYMELSSLRSEDTAVYYC,WGQGTTVTVSS,True,0,118,6.800000e-60,192.0,2.4
2,Briquilimab,Whole mAb,HeavySequence,QVQLVQSGAEVKKPGASVKVSCKASGYTFTSYNMHWVRQAPGQGLE...,imgt,True,QVQLVQSGAEVKKPGASVKVSCKASGYTFTSYNMHWVRQAPGQGLE...,human,H,IGHV1-3*01,...,QVQLVQSGAEVKKPGASVKVSCKAS,MHWVRQAPGQGLEWMGV,SYNQKFKGRVTITADKSTSTAYMELSSLRSEDTAVYYC,WGQGTLVTVSS,True,0,117,5.600000e-59,189.0,2.9
3,Narlumosbart,Whole mAb,HeavySequence,EVQLLESGGGLVQPGGSLRLSCAASGFTFSSYAMSWVRQAPGKGLE...,imgt,True,EVQLLESGGGLVQPGGSLRLSCAASGFTFSSYAMSWVRQAPGKGLE...,alpaca,H,IGHV3-23*01,...,EVQLLESGGGLVQPGGSLRLSCAAS,MSWVRQAPGKGLEWVSG,YYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYC,WGQGTLVTVSS,True,0,122,5.400000e-67,215.1,2.1
4,Varlilumab,Whole mAb,HeavySequence,QVQLVESGGGVVQPGRSLRLSCAASGFTFSSYDMHWVRQAPGKGLE...,imgt,True,QVQLVESGGGVVQPGRSLRLSCAASGFTFSSYDMHWVRQAPGKGLE...,alpaca,H,IGHV3-33*01,...,QVQLVESGGGVVQPGRSLRLSCAAS,MHWVRQAPGKGLEWVAV,YYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYC,WGQGTLVTVSS,True,0,119,2.400000e-65,209.7,2.2


In [7]:
# Display residue view 

df_result = run_anarci_toolz(
    df=df_long,
    scheme="imgt",
    allowed_species=["human"],
    seq_aa_header="sequence_aa",
    display_residue_view=True)

2025-03-11 20:06:21.067 | INFO     | anarci_toolz.abnumber_tool:run_parallel_abnumber:62 - Starting AbNumber processing ...
Processing sequences: 100%|██████████| 200/200 [00:00<00:00, 63129.20seq/s]
2025-03-11 20:06:23.404 | SUCCESS  | anarci_toolz.abnumber_tool:parallel_get_region_seqs:138 - AbNumber parallel processing complete
/Users/jonathancheng/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/anarci_toolz/numbering.py:88: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["seq_id"] = seq_id
/Users/jonathancheng/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/anarci_toolz/numbering.py:88: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider 

In [8]:
df_result.head()

,Therapeutic,Format,chain_type_input,sequence_aa,scheme,passed_abnumber,sequence_alignment_aa,species,chain_type,v_gene,...,imgt_pos_125,imgt_pos_126,imgt_pos_127,imgt_pos_128,passed_anarci,variable_region_start_index,variable_region_end_index,e_value,bitscore,bias
0,Vantictumab,Whole mAb,HeavySequence,EVQLVESGGGLVQPGGSLRLSCAASGFTFSHYTLSWVRQAPGKGLE...,imgt,True,EVQLVESGGGLVQPGGSLRLSCAASGFTFSHYTLSWVRQAPGKGLE...,alpaca,H,IGHV3-23*04,...,T,V,S,S,True,0,118,5.600000e-64,205.3,2.1
1,Sabatolimab,Whole mAb,HeavySequence,QVQLVQSGAEVKKPGSSVKVSCKASGYTFTSYNMHWVRQAPGQGLE...,imgt,True,QVQLVQSGAEVKKPGSSVKVSCKASGYTFTSYNMHWVRQAPGQGLE...,human,H,IGHV1-46*01,...,T,V,S,S,True,0,118,6.800000e-60,192.0,2.4
2,Briquilimab,Whole mAb,HeavySequence,QVQLVQSGAEVKKPGASVKVSCKASGYTFTSYNMHWVRQAPGQGLE...,imgt,True,QVQLVQSGAEVKKPGASVKVSCKASGYTFTSYNMHWVRQAPGQGLE...,human,H,IGHV1-3*01,...,T,V,S,S,True,0,117,5.600000e-59,189.0,2.9
3,Narlumosbart,Whole mAb,HeavySequence,EVQLLESGGGLVQPGGSLRLSCAASGFTFSSYAMSWVRQAPGKGLE...,imgt,True,EVQLLESGGGLVQPGGSLRLSCAASGFTFSSYAMSWVRQAPGKGLE...,alpaca,H,IGHV3-23*01,...,T,V,S,S,True,0,122,5.400000e-67,215.1,2.1
4,Varlilumab,Whole mAb,HeavySequence,QVQLVESGGGVVQPGRSLRLSCAASGFTFSSYDMHWVRQAPGKGLE...,imgt,True,QVQLVESGGGVVQPGRSLRLSCAASGFTFSSYDMHWVRQAPGKGLE...,alpaca,H,IGHV3-33*01,...,T,V,S,S,True,0,119,2.400000e-65,209.7,2.2
